<img src="https://radar.community.uaf.edu/wp-content/uploads/sites/667/2021/03/HydroSARbanner.jpg" width="100%" />
<hr>
<font size="7"> <b> Download NASADEM</b></font>

<font size="5">  Download HGT tiles from LPDAAC and Merge </font>

<br>
<font size="4"> <b> Part of NASA A.37 Project:</b> Integrating SAR Data for Improved Resilience and Response to Weather-Related Disasters   <br>
<font size="4"> <b> PI:</b>Franz J. Meyer <br>
<font size="3"> Version 0.1.0 - 2021/01/13 <br>
<b>Change Log</b><br>
See bottom of the notebook.<br>
</font> 
<font color='rgba(0,0,200,0.2)'> <b>Contact: </b> batuhan.osmanoglu@nasa.gov </font>


<hr>
<font face="Calibri">

<font size="5"> <b> 0. Importing Relevant Python Packages </b> </font>

<font size="3"> The first step in any notebook is to import the required Python libraries into the Jupyter environment. In this notebooks we use the following libraries:
<ol type="1">
    <li> <b><a href="https://www.gdal.org/" target="_blank">GDAL</a></b> is a software library for reading and writing raster and vector geospatial data formats. It includes a collection of programs tailored for geospatial data processing. Most modern GIS systems (such as ArcGIS or QGIS) use GDAL in the background.</li>
    <li> <b><a href="http://www.numpy.org/" target="_blank">NumPy</a></b> is one of the principal packages for scientific applications of Python. It is intended for processing large multidimensional arrays and matrices, and an extensive collection of high-level mathematical functions and implemented methods makes it possible to perform various operations with these objects. </li> 
    <li> <b><a href="https://docs.python.org/3/library/urllib.html" target="_blank">urllib</a></b> is an internal package that collects several modules for working with URLs.</li>
    <li> <b><a href="https://docs.python.org/3/library/zipfile.html" target="_blank">zipfile</a></b> is an internal python module provides tools to create, read, write, append, and list a ZIP file.</li>
    <li> <b><a href="https://github.com/tqdm/tqdm" target="_blank"> tqdm </a></b> is a smart progress meter that allows easy addition of a loop counter.</li>
    <li> <b><a href="https://docs.python.org/3/library/netrc.html" target="_blank">netrc</a></b> parses and encapsulates the netrc file format used by the Unix ftp program and other FTP clients. You can use to store your login by adding the line 'machine urs.earthdata.nasa.gov login YOUR_USERNAME password YOUR_PASSWORD'</li>
    <li> <b><a href="https://docs.python.org/3/library/http.cookiejar.html" target="_blank">http.cookiejar</a></b> defines classes for automatic handling of HTTP/HTTPS cookies.</li>
    <li> <b><a href="https://docs.python.org/3/library/getpass.html" target="_blank">getpass</a></b> is used to prompt the user for a password without echoing on screen.</li>
    

In [ ]:
# Setup Environment
import sys
import os
from tqdm.auto import tqdm
from osgeo import osr
from osgeo import gdal
import pyproj
import zipfile
from getpass import getpass  # used to input URS creds and add to .netrc
import urllib
import numpy as np
import time #for sleep
import netrc
import ipywidgets as ui
from IPython.display import display
try:
    from http.cookiejar import CookieJar
except ImportError:
    from cookielib import CookieJar


<font size="5"> <b> 1. Define convenience functions </b> </font>

<font size="3"> Here we define some functions for later convenience.
    
<ol type="1">
    <li> <b>build_vrt</b> generates a virtual raster (VRT) file with gdal from a list of raster files. </li>
    <li> <b>download_nasadem_tile</b> downloads a single NASADEM tile after logging in to LPDAAC.</li>
    <li> <b>download_nasadem</b> is the interface to download multiple tiles and merge them as a VRT file. </li>
    <li> <b>yesno</b> allows the user to respond to a question with simple yes/no text input. </li>         

In [ ]:
# Define Functions
def numel(x):
    if isinstance(x, np.int):
      return 1
    elif isinstance(x, np.double):
      return 1
    elif isinstance(x, np.float):
      return 1
    elif isinstance(x, str):
      return 1
    elif isinstance(x, list) or isinstance(x, tuple):
      return len(x)
    elif isinstance(x, np.ndarray):
      return x.size
    else: 
      print('Unknown type {}.'.format(type(x)))
      return None

def gdal_get_geotransform(filename):
    '''
    [top left x, w-e pixel resolution, rotation, top left y, rotation, n-s pixel resolution]=gdal_get_geotransform('/path/to/file')
    '''
    #http://stackoverflow.com/questions/2922532/obtain-latitude-and-longitude-from-a-geotiff-file
    ds = gdal.Open(filename)
    return ds.GetGeoTransform()

def gdal_get_projection(filename, out_format='proj4'):
    """
    epsg_string=get_epsg(filename, out_format='proj4')
    """
    try:
      ds=gdal.Open(filename, gdal.GA_ReadOnly)
      srs=gdal.osr.SpatialReference()
      srs.ImportFromWkt(ds.GetProjectionRef())
    except: #I am not sure if this is working for datasets without a layer. The first try block should work mostly.
      ds=gdal.Open(filename, gdal.GA_ReadOnly)
      ly=ds.GetLayer()
      if ly is None:
        print(f"Can not read projection from file:{filename}")
        return None
      else:
        srs=ly.GetSpatialRef()
    if out_format.lower()=='proj4':
      return srs.ExportToProj4()
    elif out_format.lower()=='wkt':
      return srs.ExportToWkt()
    elif out_format.lower()=='epsg':
      crs=pyproj.crs.CRS.from_proj4(srs.ExportToProj4())
      return crs.to_epsg()

def gdal_get_WESN(filename):
    '''
    (minx,miny,maxx,maxy)=corners('/path/to/file')
    '''
    #http://stackoverflow.com/questions/2922532/obtain-latitude-and-longitude-from-a-geotiff-file
    ds = gdal.Open(filename)
    width = ds.RasterXSize
    height = ds.RasterYSize
    gt = ds.GetGeoTransform()
    minx = gt[0]
    miny = gt[3] + width*gt[4] + height*gt[5] 
    maxx = gt[0] + width*gt[1] + height*gt[2]
    maxy = gt[3] 
    return (minx,maxx,miny,maxy) #(minx,miny,maxx,maxy)    

def transform_point(x,y,z,s_srs='+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs', t_srs='+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs'):
    '''
    transform_point(x,y,z,s_srs='+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs', t_srs='+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs')
    
    Known Bugs: gdal transform may fail if a proj4 string can not be found for the EPSG or WKT formats. 
    '''    
    srs_cs=osr.SpatialReference()    
    if "EPSG" == s_srs[0:4]:    
      srs_cs.ImportFromEPSG(int(s_srs.split(':')[1]));
    elif "GEOCCS" == s_srs[0:6]:
      srs_cs.ImportFromWkt(s_srs);
    else:
      srs_cs.ImportFromProj4(s_srs);

    trs_cs=osr.SpatialReference()    
    if "EPSG" == t_srs[0:4]:    
      trs_cs.ImportFromEPSG(int(t_srs.split(':')[1]));
    elif "GEOCCS" == t_srs[0:6]:
      trs_cs.ImportFromWkt(t_srs);
    else:
      trs_cs.ImportFromProj4(t_srs);
    transform = osr.CoordinateTransformation(srs_cs,trs_cs) 
    
    if numel(x)>1:
      return [  transformPoint(x[k], y[k], z[k]) for k in range(numel(x))]
    else:
      try:
        return transform.TransformPoint((x,y,z));
      except: 
        return transform.TransformPoint(x,y,z)

def warp(src_filename, dst_filename, pixel_spacing=0.00008333, xRes=None, yRes=None, resampleAlg='nearest', dstSRS="EPSG:4326", tps=False, rpc=False):
    if xRes is None and pixel_spacing:
      xRes=pixel_spacing
    if yRes is None and pixel_spacing:
      yRes=pixel_spacing
    t=TqdmUpTo()
    gwo=gdal.WarpOptions(xRes=xRes, yRes=yRes, resampleAlg=resampleAlg, dstSRS=dstSRS, callback=t.callback)
    gdal.Warp(dst_filename, src_filename, options=gwo)
    del t

def build_vrt(filename, input_file_list, targetAlignedPixels=True, separate=False, resampleAlg='near', resolution='highest'):
    vrt_options = gdal.BuildVRTOptions(resampleAlg=resampleAlg, resolution=resolution, separate=separate, targetAlignedPixels=targetAlignedPixels)
    ds=gdal.BuildVRT(filename,input_file_list,options=vrt_options)
    ds.FlushCache()
        
def download_nasadem_tile(tile, url="https://e4ftl01.cr.usgs.gov/MEASURES", version="NASADEM_HGT.001", username=None, password=None, download_folder=None, debug=False):
    #Modified from: https://github.com/OSGeo/grass-addons/blob/master/grass7/raster/r.in.nasadem/r.in.nasadem.py
    if download_folder is None:
        download_folder = os.getcwd()
    if not username or not password:
        print(f"Enter your NASA EarthData username:")
        username = input()
        print(f"Enter your password:")
        password = getpass()
    
    if debug: print("Download tile: %s" % tile)
    local_tile = "NASADEM_HGT_" + str(tile) + ".zip"
    output_path=os.path.join(download_folder,local_tile)

    urllib.request.urlcleanup()

    remote_tile = str(url) + "/" + version + "/2000.02.11/" + local_tile
    goturl = 1

    try:
        password_manager = urllib.request.HTTPPasswordMgrWithDefaultRealm()
        password_manager.add_password(
            None, "https://urs.earthdata.nasa.gov", username, password
        )

        cookie_jar = CookieJar()
        if debug:
            opener = urllib.request.build_opener(
                urllib.request.HTTPBasicAuthHandler(password_manager),
                urllib.request.HTTPHandler(debuglevel=1),    # Uncomment these two lines to see
                urllib.request.HTTPSHandler(debuglevel=1),   # details of the requests/responses
                urllib.request.HTTPCookieProcessor(cookie_jar),
            )
        else:
            opener = urllib.request.build_opener(
                urllib.request.HTTPBasicAuthHandler(password_manager),
                urllib.request.HTTPCookieProcessor(cookie_jar),
            )            
        urllib.request.install_opener(opener)
        request = urllib.request.Request(remote_tile)
        response = urllib.request.urlopen(request)
        if os.path.exists(output_path):
            print(f'Overwriting existing file:{output_path}')
        fo = open(output_path, "w+b")
        fo.write(response.read())
        fo.close
        time.sleep(0.5)
    except FileNotFoundError:
        print(f'Can not create file: {output_path}')
        goturl = 0
        pass        
    except:
        goturl = 0
        pass

    return goturl

def download_nasadem(W,E,S,N, url="https://e4ftl01.cr.usgs.gov/MEASURES", version="NASADEM_HGT.001", download_path=None, debug=False, keep_downloads=False, username=None, password=None):
    if not username or not password:
        parts = urllib.parse.urlparse(url)
        try:
            username, account, password = netrc.netrc().authenticators(parts.netloc)
        except:
            print(f"Enter your NASA EarthData username:")
            username = input()
            print(f"Enter your password:")
            password = getpass()
        
    if download_path is not None:
        download_folder=os.path.dirname(download_path)
        output_file=download_path
    else:
        download_folder=os.getcwd()
        output_file='nasadem.vrt'    
    
    north=int(np.ceil(N))
    south=int(np.floor(S))
    west =int(np.floor(W))
    east =int(np.ceil(E))
    rows = abs(north - south)
    cols = abs(east - west)
    ntiles = rows * cols
    if debug: print("Importing %d NASADEM tiles..." % ntiles)

    zip_files = []
    hgt_files = []
    for ndeg in tqdm(range(south, north)):
        for edeg in tqdm(range(west, east)):
            if ndeg < 0:
                tile = "s"
            else:
                tile = "n"
            tile = tile + "%02d" % abs(ndeg)
            if edeg < 0:
                tile = tile + "w"
            else:
                tile = tile + "e"
            tile = tile + "%03d" % abs(edeg)
            if debug: print("Tile: %s" % tile)

            local_tile = "NASADEM_HGT_" + str(tile) + ".zip"
            tile_name= "NASADEM_HGT_" + str(tile)
            local_hgt  = os.path.join(download_folder, tile_name, tile_name + ".hgt")            
            if os.path.isfile(os.path.join(download_folder,local_tile)):  #is the zip folder there?  
                if os.path.isfile(local_hgt): #yes, but is the hgt file there?
                    hgt_files.append(local_hgt) #if so add hgt_file to list
                else: # no hgt so add zip file to list.
                    zip_files.append(os.path.join(download_folder,local_tile))
            else: #zip file is not there
                if os.path.isfile(local_hgt): #if not is the hgt file there?
                    hgt_files.append(local_hgt) #if so add hgt_file to list
                else: #if not download
                    success=download_nasadem_tile(tile, url=url, version=version, 
                              username=username, password=password, 
                                      download_folder=download_folder, debug=debug)                                            
                    if success:
                        zip_files.append(os.path.join(download_folder,local_tile))
                    else:
                        if debug: print(f'Missing/Water tile: {tile}') 
                            
    if debug: print(f'Zip_files: {zip_files}')
    if debug: print(f'hgt_files: {hgt_files}')        
    #start_splicing  
    print('Unzipping...')
    zip_contents=[]
    #extract_folders=[os.path.splitext(f)[0] for f in zip_files]
    for f in zip_files:#zip(zip_files, extract_folders):        
        with zipfile.ZipFile(f, 'r') as zip_ref:  
            zip_contents.append(zip_ref.namelist())
            zip_ref.extractall(path=download_folder)
    if debug: print(zip_contents)
    #convert zip_contents to list of full paths.
    vrt_contents=[ os.path.join(download_folder,os.path.splitext(zip_contents[k][0])[0] + '.hgt') for k in range(len(zip_files))]
    vrt_contents.extend(hgt_files)                            
    if debug: print(f'vrt_contents:{vrt_contents}')
    if len(vrt_contents)>0:
        #combine with gdal
        build_vrt(output_file, vrt_contents, targetAlignedPixels=False) #targetAlignedPixels has to be False. Otherwise no file is created for some reason. 
        print(f'Successfully generated:{output_file}')                
    else:
        print('No tiles found.')
    #cleanup
    if debug or keep_downloads:
        print('Skipping cleanup in debug mode or when keep_downloads is set.')
        print(f'Files NOT deleted: {zip_files}')        
    else:
        for f in zip_files:
            os.remove(f)

def yesno(yes_no_question="[y/n]"):
    while True:
        # raw_input returns the empty string for "enter"
        yes = {'yes','y', 'ye'}
        no = {'no','n'}

        choice = input(yes_no_question+"[y/n]").lower()
        if choice in yes:
            return True
        elif choice in no:
            return False
        else:
            print("Please respond with 'yes' or 'no'")    
            
class TqdmUpTo(tqdm): #Used in warp()
    """Provides `update_to(n)` which uses `tqdm.update(delta_n)`."""
    def update_to(self, b=1, bsize=1, tsize=None):
        """
        b  : int, optional
            Number of blocks transferred so far [default: 1].
        bsize  : int, optional
            Size of each block (in tqdm units) [default: 1].
        tsize  : int, optional
            Total size (in tqdm units). If [default: None] remains unchanged.
        """
        if tsize is not None:
            self.total = tsize
        return self.update(b * bsize - self.n)  # also sets self.n = b * bsize
    def callback(self,complete, message, data):
        percent = int(complete * 100)  # round to integer percent
        self.update_to(percent, tsize=100)
        
class PathSelector():
    """
    Displays a file selection tree. Any file can be selected. 
    Selected path can be obtained by: PathSelector.accord.get_title(0)
    """
    def __init__(self,start_dir,select_file=True):
        self.file        = None 
        self.select_file = select_file
        self.cwd         = start_dir
        self.select      = ui.SelectMultiple(options=['init'],value=(),rows=10,description='') 
        self.accord      = ui.Accordion(children=[self.select]) 

        self.accord.selected_index = None # Start closed (showing path only)
        self.refresh(self.cwd)
        self.select.observe(self.on_update,'value')

    def on_update(self,change):
        if len(change['new']) > 0:
            self.refresh(change['new'][0])

    def refresh(self,item):
        path = os.path.abspath(os.path.join(self.cwd,item))

        if os.path.isfile(path):
            if self.select_file:
                self.accord.set_title(0,path)  
                self.file = path
                self.accord.selected_index = None
            else:
                self.select.value = ()

        else: # os.path.isdir(path)
            self.file = None 
            self.cwd  = path

            # Build list of files and dirs
            keys = ['[..]']; 
            for item in os.listdir(path):
                if item[0] == '.':
                    continue
                elif os.path.isdir(os.path.join(path,item)):
                    keys.append('['+item+']'); 
                else:
                    keys.append(item); 

            # Sort and create list of output values
            keys.sort(key=str.lower)
            vals = []
            for k in keys:
                if k[0] == '[':
                    vals.append(k[1:-1]) # strip off brackets
                else:
                    vals.append(k)

            # Update widget
            self.accord.set_title(0,path)  
            self.select.options = list(zip(keys,vals)) 
            with self.select.hold_trait_notifications():
                self.select.value = ()            

In [ ]:
# Define Input Parameters

## 1. Bounding Box for the Area of Interest ##
W=-122.9#-96.5001618 #upper left
N=45.9#41.5002284
E=-121.0#-95.1003219 #lower right
S=45.0#39.9001804
#VA
S=37.6;N=37.96;W=-76.6;E=-76.2
#Miami Beach
S=25.8;N=25.9;W=-80.2;E=-80.1
## 2. Output Virtual Raster (VRT) file name ##
output_VRT=f'~/NASADEM/N{int(N):02}_S{int(S):02}_W{int(W):03}_E{int(E):03}.vrt' #All HGT files will be stored next to the VRT file.
debug=False          #Set True to turn on verbose output
keep_zip_files=False #By default downloaded zip files are deleted. 

In [ ]:
# Select gdal file if WESN is all zeros. 
if W==E or N==S:
    print("Choose your GDAL compatible file using the file browser below:")
    f = PathSelector('.')
    display(f.accord)    
else:
    pass

In [ ]:
#Get W,E,S,N if needed. 
if W==E or N==S:
    gdal_file=f.accord.get_title(0)    
    if os.path.exists(gdal_file):
        print(f'Selected file: {gdal_file}')
    else:
        print(f'Can not find file: {gdal_file}')
        raise ValueError
    W,E,S,N=gdal_get_WESN(gdal_file)
    epsg=gdal_get_projection(gdal_file, out_format='epsg')
    if epsg=="4326":
        pass
    else:
        srs=gdal_get_projection(gdal_file, out_format='proj4')
        W,N,h=transform_point(W,N,0,s_srs=srs)
        E,S,h=transform_point(E,S,0,s_srs=srs)
        W=np.round(W,2)
        E=np.round(E,2)
        S=np.round(S,2)
        N=np.round(N,2)
        del h # we don't use height
    print(f'Bounding Box W/E/S/N: {W} / {E} / {S} / {N}')

In [ ]:
# Download and stitch tiles 
output_VRT=os.path.expanduser(output_VRT) #expand ~ to user home
if os.path.exists(output_VRT):
    if yesno(f"Overwrite file: {output_VRT}"):
        pass
    else:
        assert False
download_nasadem(W,E,S,N, download_path=output_VRT, debug=debug, keep_downloads=keep_zip_files)

<font face="Calibri" size="2" color="gray"> <i> Version 0.1.0 - Batu Osmanoglu
    
<b>Change Log</b> <br>
2021/01/13: v0.1.0 <br>
-Initial version.<br>